Предсказание длинны очереди

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
import os.path
filename =  "data_processed.csv"
def get_or_create(filename, generate_dataset_function = None, recreate=None):
    if filename.endswith('.gz'):
        compression = 'gzip'
    elif filename.endswith('.csv'):
        compression = None
    else:
        raise AttributeError("Check filename")
    
    if not os.path.exists(filename) or recreate == True:
        df = generate_dataset_function()
        df.to_csv(filename, compression=compression)
    else:
        df = pd.read_csv(filename, compression=compression)
        df.drop(columns=["Unnamed: 0"], inplace=True)
    return df

In [3]:
df = get_or_create(filename)
df.describe()

,clients_count,queue_length_mean,queue_length_sd,serving_time_mean,serving_time_sd
count,238.000000,238.000000,238.000000,238.000000,238.000000
mean,13.554622,4.331076,3.104534,286.003008,124.954177
std,4.468733,3.431805,1.796154,161.902989,93.515307
min,5.000000,0.333333,0.471536,120.000000,0.000000
25%,11.000000,1.722222,1.630206,178.234848,63.454816
50%,13.000000,3.178333,2.659807,229.363636,98.328767
75%,16.000000,6.125972,4.408904,342.189732,150.548063
max,28.000000,16.753333,8.563672,1042.000000,493.846401


In [4]:
# # mu = 200
# # sd = 120
# # size = 60*10
# array = np.random.normal(200, 100, 60*10)
# x = array.reshape(1, -1)
# x = array.reshape(-1, 1)

# density = np.histogram(array, density=True)

# from sklearn.neighbors.kde import KernelDensity

# kd = KernelDensity().fit(x)
# kd.score_samples(x)

# start = 1
# end = 60*10



In [5]:
from scipy.stats import gaussian_kde
array = np.random.normal(250, 100, 60*10)
kde = gaussian_kde(array)

pdf_df = pd.DataFrame(columns=["value", "probability"])

for i in range(1,600):
    pdf_df = pdf_df.append({"value": i, "probability": kde.pdf(i)[0]}, ignore_index=True)
#pdf_df.as_matrix
#pdf_df.values
#kde.integrate_box_1d(1,600)
#kde.pdf(350)


In [6]:
#pdf_df.query("value < 120")["probability"]
#pdf_df.iloc[120:,]["probability"] + pdf_df.query("value < 120")["probability"]

In [7]:
df = get_or_create(filename)
df.describe()

,clients_count,queue_length_mean,queue_length_sd,serving_time_mean,serving_time_sd
count,238.000000,238.000000,238.000000,238.000000,238.000000
mean,13.554622,4.331076,3.104534,286.003008,124.954177
std,4.468733,3.431805,1.796154,161.902989,93.515307
min,5.000000,0.333333,0.471536,120.000000,0.000000
25%,11.000000,1.722222,1.630206,178.234848,63.454816
50%,13.000000,3.178333,2.659807,229.363636,98.328767
75%,16.000000,6.125972,4.408904,342.189732,150.548063
max,28.000000,16.753333,8.563672,1042.000000,493.846401


In [8]:
pdf_df.describe()

,value,probability
count,599.000000,599.000000
mean,300.000000,0.001641
std,173.060683,0.001405
min,1.000000,0.000007
25%,150.500000,0.000377
50%,300.000000,0.001193
75%,449.500000,0.002848
max,599.000000,0.004304


# Features

In [9]:
def prepare_features(X):
    from featuretools.primitives import Minute, Hour, Day, Month, Year, Weekday

    import featuretools as ft

    es = ft.EntitySet("mm")

    from featuretools import variable_types as vtypes

    variable_types = {
        'time': vtypes.Datetime,
        #'clients_count': vtypes.Numeric,

    }

    es.entity_from_dataframe(entity_id="chunks",
                             dataframe=X,
                             index="index",
                             time_index='time',
                             variable_types=variable_types)

    feature_matrix, feature_defs = ft.dfs(entityset=es, target_entity="chunks", trans_primitives=[Minute, Hour, Day, Weekday])
    return feature_matrix

# Train models

In [10]:
y = df[["queue_length_mean", "queue_length_sd"]]
X = df.drop(y.columns, axis=1).reset_index()
X = prepare_features(X)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import mean_squared_error

from sklearn.pipeline import Pipeline

from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [13]:
## Predict queue_length_mean and queue_length_sd apart

In [14]:
#y1_train = y_train.queue_length_mean
#y1_test = y_test.queue_length_mean

In [15]:
pipeline = Pipeline([
    #('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
    ('model',RandomForestRegressor())
])
scores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring='neg_mean_squared_error')

pipeline.fit(X_train,y_train)

y_preds = pipeline.predict(X_test)

metric = mean_squared_error(y_test, y_preds)
print("Metric for queue_length_mean: %s" % metric)

Metric for queue_length_mean: 1.087231200326944


In [16]:
import datetime

In [17]:
def predict_wrapper(numer_of_clients, duration_density = pdf_df, time = None):
    global pipeline
    def convert_density(duration_density):
        mean = duration_density["probability"].max()
        real_mean = duration_density.iloc[(duration_density['probability']-mean).abs().argsort()[:1]].value.mean()
        
        sd = duration_density["probability"].std()
        real_sd = duration_density.iloc[(duration_density['probability']-sd).abs().argsort()[:1]].value.mean()
        real_sd = abs(real_mean-real_sd)
        
        return real_mean, real_sd
    real_mean, real_sd = convert_density(duration_density)
    raw = pd.DataFrame([{"clients_count": numer_of_clients, "serving_time_mean": real_mean, "serving_time_sd": real_sd, "time": time}])
    x = prepare_features(raw.reset_index())
    return pipeline.predict(x)
    
predict_wrapper(15, duration_density = pdf_df, time=datetime.datetime(2018, 1, 1, 7, 0))

array([[2.95538889, 2.51062023]])